<a href="https://colab.research.google.com/github/Crunza17/Colab.ChatBot/blob/main/Indexer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Conecta Colab con Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define la ruta
PROJECT_ROOT = "/content/drive/MyDrive/Proyecto"

# Define las rutas que usaremos
KNOWLEDGE_BASE_DIR = f"{PROJECT_ROOT}/Conjunto_de_datos"
VECTOR_STORE_DIR = f"{PROJECT_ROOT}/vector_store"

print(f"Ruta de la base de conocimiento: {KNOWLEDGE_BASE_DIR}")
print(f"Ruta donde se guardará la base de datos vectorial: {VECTOR_STORE_DIR}")

In [ ]:
!pip install -q --upgrade langchain langchain-community langchain-google-genai langchain-text-splitters
!pip install -q --upgrade pypdf python-dotenv sentence-transformers unstructured
!pip install -q --upgrade chromadb pysqlite3-binary

In [ ]:
!pip install -q --upgrade langchain langchain-community langchain-google-genai langchain-text-splitters

In [ ]:
from google.colab import userdata
import os

try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("✅ API Key de Google configurada correctamente.")
except Exception as e:
    print("⚠️ Error: No se pudo encontrar la API Key. Asegúrate de haberla guardado en los 'Secretos' con el nombre GOOGLE_API_KEY.")

In [4]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
# -------------------------------------------------------------------------

from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
import os

def index_documents():
    """Procesa y almacena los documentos en una base de datos vectorial."""
    print("Iniciando la carga de documentos...")

    if not os.path.exists(KNOWLEDGE_BASE_DIR):
        print(f"Error: No existe la ruta {KNOWLEDGE_BASE_DIR}")
        return

    txt_loader = DirectoryLoader(KNOWLEDGE_BASE_DIR, glob="**/*.txt", show_progress=True)
    pdf_loader = DirectoryLoader(KNOWLEDGE_BASE_DIR, glob="**/*.pdf", loader_cls=PyPDFLoader, show_progress=True)

    try:
        documents = txt_loader.load() + pdf_loader.load()
    except Exception as e:
        print(f"Error cargando documentos: {e}")
        return

    if not documents:
        print("¡Advertencia! No se encontraron documentos en la carpeta.")
        return

    print("Dividiendo documentos en fragmentos (chunks)...")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)
    print(f"Los documentos se dividieron en {len(texts)} fragmentos.")

    print("Creando embeddings con un modelo local...")
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)

    print("Inicializando la base de datos vectorial en ChromaDB...")

    # Creamos la base de datos persistente
    vector_store = Chroma(
        persist_directory=VECTOR_STORE_DIR,
        embedding_function=embeddings,
        collection_name="conjunto_datos_vectorizado"
    )

    batch_size = 166  # Reducido un poco para evitar límites de memoria en Colab gratuito
    total_chunks = len(texts)

    print(f"Añadiendo {total_chunks} fragmentos a la base de datos...")

    # Procesamiento por lotes
    for i in range(0, total_chunks, batch_size):
        batch = texts[i:i + batch_size]
        vector_store.add_documents(documents=batch)
        print(f"  -> Lote procesado: {min(i + batch_size, total_chunks)} / {total_chunks}")

    # En versiones nuevas de Chroma, persist() suele ser automático, pero lo dejamos por seguridad si la versión lo soporta
    try:
        vector_store.persist()
        print("Persistiendo cambios...")
    except:
        pass # En versiones muy nuevas persist() ya no es necesario ni existe

    print("\n¡Proceso de indexación completado exitosamente!")
    print(f"Tu base de datos vectorial está guardada en: {VECTOR_STORE_DIR}")

In [5]:
# Llama a la función
index_documents()

Iniciando la carga de documentos...


100%|██████████| 4/4 [04:38<00:00, 69.54s/it]


Dividiendo documentos en fragmentos (chunks)...
Los documentos se dividieron en 19520 fragmentos.
Creando embeddings con un modelo local...


/tmp/ipython-input-3541297936.py:41: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Inicializando la base de datos vectorial en ChromaDB...


/tmp/ipython-input-3541297936.py:46: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_store = Chroma(


Añadiendo 19520 fragmentos a la base de datos...
  -> Lote procesado: 166 / 19520
  -> Lote procesado: 332 / 19520
  -> Lote procesado: 498 / 19520
  -> Lote procesado: 664 / 19520
  -> Lote procesado: 830 / 19520
  -> Lote procesado: 996 / 19520
  -> Lote procesado: 1162 / 19520
  -> Lote procesado: 1328 / 19520
  -> Lote procesado: 1494 / 19520
  -> Lote procesado: 1660 / 19520
  -> Lote procesado: 1826 / 19520
  -> Lote procesado: 1992 / 19520
  -> Lote procesado: 2158 / 19520
  -> Lote procesado: 2324 / 19520
  -> Lote procesado: 2490 / 19520
  -> Lote procesado: 2656 / 19520
  -> Lote procesado: 2822 / 19520
  -> Lote procesado: 2988 / 19520
  -> Lote procesado: 3154 / 19520
  -> Lote procesado: 3320 / 19520
  -> Lote procesado: 3486 / 19520
  -> Lote procesado: 3652 / 19520
  -> Lote procesado: 3818 / 19520
  -> Lote procesado: 3984 / 19520
  -> Lote procesado: 4150 / 19520
  -> Lote procesado: 4316 / 19520
  -> Lote procesado: 4482 / 19520
  -> Lote procesado: 4648 / 19520
  -> 

/tmp/ipython-input-3541297936.py:65: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [6]:
# Fuerza la sincronización de todos los archivos pendientes con Google Drive.

from google.colab import drive
import time

print("\n⏳ Forzando la sincronización final con Google Drive...")

# Esta función especial "vacía" el caché y fuerza la escritura en Drive.
drive.flush_and_unmount()
print("\n✅ Sincronización completada.")

# Volvemos a montar Drive para que puedas seguir usando el notebook si lo necesitas.
time.sleep(5)
drive.mount('/content/drive')


⏳ Forzando la sincronización final con Google Drive...

✅ Sincronización completada.
Mounted at /content/drive
